In [83]:
import os


In [84]:
pwd

'c:\\Users\\vansh\\AI and ML reading material\\MLOps\\New folder\\Project 1-Chest Cancer prediction model'

In [85]:
os.chdir('../')


In [86]:
pwd

'c:\\Users\\vansh\\AI and ML reading material\\MLOps\\New folder'

In [87]:
os.chdir('c:\\Users\\vansh\\AI and ML reading material\\MLOps\\New folder\\Project 1-Chest Cancer prediction model')

In [88]:
from cnnClassifier.constants import *
yaml_path = CONFIG_FILE_PATH
params = PARAMS_FILE_PATH
print(yaml_path)
print(yaml_path.resolve())
print(yaml_path.exists())

print(params)
print(params.resolve())
print(params.exists())

C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\config\config.yaml
C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\config\config.yaml
True
C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\params.yaml
C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\params.yaml
True


In [89]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [90]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:

    
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )   

        return data_ingestion_config

        

In [91]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [92]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id=' #This is the url format to download any file from google drive.
            gdown.download(prefix+file_id,zip_download_dir) #we have to add the file_id with the prefix.

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """Extracts zip file into data directory"""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file) as zip_ref:
            zip_ref.extractall(unzip_path)
            
            


In [93]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-08-13 13:33:44,078: INFO: common: yaml file: C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\config\config.yaml load successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ucuX5BjM-DBkSRfhMooyUqW3A7PixUZ3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-08-13 13:33:44,079: INFO: common: yaml file: C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\params.yaml load successfully]
{'key': 'val'}
[2024-08-13 13:33:44,081: INFO: common: created directory at :artifacts]
[2024-08-13 13:33:44,082: INFO: common: created directory at :artifacts/data_ingestion]
[2024-08-13 13:33:44,082: INFO: 187801436: Downloading data from https://drive.google.com/file/d/1ucuX5BjM-DBkSRfhMooyUqW3A7PixUZ3/view?usp=sharing into file artifact

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1ucuX5BjM-DBkSRfhMooyUqW3A7PixUZ3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1ucuX5BjM-DBkSRfhMooyUqW3A7PixUZ3&confirm=t&uuid=cada4abe-2198-41c0-b909-fa3886996052
To: c:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\artifacts\data_ingestion\data.zip
100%|██████████| 121M/121M [00:26<00:00, 4.55MB/s] 


[2024-08-13 13:34:16,762: INFO: 187801436: Downloaded data from https://drive.google.com/file/d/1ucuX5BjM-DBkSRfhMooyUqW3A7PixUZ3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
